In [ ]:
pip install llama-index-core llama-index-embeddings-ollama llama-index-llms-ollama

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [5]:
# Modelos locais via Ollama
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
llm = Ollama(model="llama3:8b")

In [6]:
# Carregar documentos (ex.: PDFs em ./data)
docs = SimpleDirectoryReader("data").load_data()

In [7]:
# Criar índice e query engine
index = VectorStoreIndex.from_documents(docs, 
                                        embed_model=embed_model)
qe = index.as_query_engine(llm=llm)

In [8]:
# Perguntar
resp = qe.query("Quais os princípios do SUS segundo a Lei 8.080?")
print(resp)

Segundo a Lei nº 8.080/90, os princípios do Sistema Único de Saúde (SUS) são:

I - Universalidade: o direito à saúde é universal e igual para todos;

II - Integralidade: a atenção à saúde deve ser integral, considerando a multidisciplinaridade e a complementariedade das ações e serviços;

III - Equidade: o acesso à saúde deve ser equitativo e não discriminatório;

IV - Comunitário: a atenção à saúde deve ser desenvolvida em consonância com as necessidades e realidades locais;

V - Participação: a sociedade civil, os usuários e os profissionais de saúde devem participar do processo decisório e de controle sobre o sistema de saúde.

Esses princípios constituem a base do SUS e orientam sua organização e funcionamento.


## LLAMAINDEX com chunk explícito

In [54]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Modelo local via Ollama
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
llm = Ollama(model="llama3:8b")

# Ler vários arquivos de uma pasta (PDF/TXT/DOCX…)
docs = SimpleDirectoryReader("data").load_data()

# CHUNKING: dividir em pedaços menores (granularidade controlada)
splitter = SentenceSplitter(chunk_size=800, chunk_overlap=120)
nodes = splitter.get_nodes_from_documents(docs)  # vários chunks -> 1 embedding por chunk

# Indexar (cada chunk vira um vetor)
index = VectorStoreIndex(nodes, embed_model=embed_model)

# Perguntar: só top-k trechos entram no prompt (não o corpus inteiro)
qe = index.as_query_engine(llm=llm, similarity_top_k=3)
resp = qe.query("Quais são os princípios do SUS segundo a Lei 8.080?")
print(resp)

According to the provided context information, the principles of the SUS (Sistema Único de Saúde) according to Lei 8.080 are not explicitly stated. However, it is likely that they can be found within the contents of the file_path "c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\lei-8080.pdf".


### Melhorando o RAG no LLAMA INDEX

In [ ]:
#pip install llama-index-core llama-index-embeddings-ollama llama-index-llms-ollama pymupdf
#!pip install llama-index-readers-file pymupdf

Defaulting to user installation because normal site-packages is not writeable

   -------------------------- ------------- 2/3 [llama-index-readers-file]
   -------------------------- ------------- 2/3 [llama-index-readers-file]
   ---------------------------------------- 3/3 [llama-index-readers-file]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\ander\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [84]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.prompts import PromptTemplate
from llama_index.readers.file import PyMuPDFReader

In [83]:
# Modelos locais
embed = OllamaEmbedding(model_name="nomic-embed-text")
llm = Ollama(model="llama3:8b")

In [85]:
#Carregar documentos (ex.: ./data com PDFs/TXTs)
docs = PyMuPDFReader().load("data/lei-8080.pdf")

In [87]:
# CHUNKING explícito (melhora granularidade)
splitter = SentenceSplitter(chunk_size=800, chunk_overlap=120)
nodes = splitter.get_nodes_from_documents(docs)

In [89]:
# Indexar (em memória)
index = VectorStoreIndex(nodes, embed_model=embed)

2025-09-01 21:42:53,299 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,332 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,365 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,386 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,422 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,451 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,474 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,497 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,526 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:42:53,570 - INFO - HTTP Request: POST http://localhost:1143

In [91]:
# PROMPT mais elaborado (citar fonte / permitir “Não sei”)
qa_tmpl = PromptTemplate(
    "Responda APENAS com base nos trechos fornecidos.\n"
    "Se não houver evidência suficiente, diga 'Não sei'.\n\n"
    "Pergunta: {query_str}\n"
    "Trechos:\n{context_str}\n"
)

In [100]:
# Pós-processador para cortar irrelevantes por similaridade
post = [SimilarityPostprocessor(similarity_cutoff=0.60)]

In [ ]:
# Query engine com top-k pequeno + cutoff + prompt próprio
qe = index.as_query_engine(
    llm=llm,
    similarity_top_k=10,
    text_qa_template=qa_tmpl,
    node_postprocessors=post,  # descarta trechos fracos
    response_mode="compact",   # resposta objetiva
)

In [104]:
# Perguntar e mostrar fontes
q = "Quais são os princípios do SUS?"
resp = qe.query(q)
print(resp)  # resposta já sintetizada

2025-09-01 21:45:12,568 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-09-01 21:45:16,495 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Os princípios do SUS são:

I - universalidade de acesso aos serviços de saúde em todos os níveis de assistência;

II - decisão sobre os aspectos operacionais, financeiros e administrativos da gestão compartilhada do SUS, em conformidade com a definição da política consubstanciada em planos de saúde, aprovados pelos conselhos de saúde.

Não há mais trechos que mencionem outros princípios.


In [103]:
print("\nFontes:")
for n in resp.source_nodes:
    print("-", n.node.metadata.get("file_path", n.node.metadata.get("filename", "")))


Fontes:
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
- data/lei-8080.pdf
